# FOREST COVERTYPE DATA

## Importaciones

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
print('Setup complete')

Setup complete


## Creación del dataset

In [29]:
df_bosque = pd.read_csv('data/covtype.data', header=None)

In [30]:
columnas = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
wilderness_columns = []
soil_columns = []
for i in range(1, 5):
    wilderness_columns.append('Wilderness_Area_' + str(i))
for i in range(1, 41):
    soil_columns.append('Soil_Type_' + str(i))
columnas += wilderness_columns
columnas += soil_columns
columnas.append('Cover_Type')
df_bosque.columns = columnas
df_bosque.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


**JUNTAR LAS COLUMNAS ÁREAS SILVESTRES Y TIPOS DE SUELO**

En estas columnas se encuentra en código binario, siendo 0 la presencia y 1 si hay. Por tanto, sumaremos los unos y juntarlos en una columna para saber cuántos tipos hay.

In [31]:
df_bosque['Wilderness_Area'] = df_bosque[wilderness_columns].max(axis=1)
df_bosque.drop(wilderness_columns, axis=1, inplace=True)
df_bosque['Soil_Type'] = df_bosque[soil_columns].max(axis=1)
df_bosque.drop(soil_columns, axis=1, inplace=True)
df_bosque.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Wilderness_Area,Soil_Type
0,2596,51,3,258,0,510,221,232,148,6279,5,1,1
1,2590,56,2,212,-6,390,220,235,151,6225,5,1,1
2,2804,139,9,268,65,3180,234,238,135,6121,2,1,1
3,2785,155,18,242,118,3090,238,238,122,6211,2,1,1
4,2595,45,2,153,-1,391,220,234,150,6172,5,1,1


In [32]:
df_bosque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Cover_Type                          581012 non-null  int64
 11  Wilderness_Area                     581012 non-null 

### Preparación del dataset

Comprobamos si hay valores nulos o algunos de ellos están repetidos.

In [33]:
nulos = df_bosque.isna().sum()
nulos

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Cover_Type                            0
Wilderness_Area                       0
Soil_Type                             0
dtype: int64

Ya hemos visto que no se encuentran. Falta comprobar si hay filas duplicadas, en el caso de que las hubiese se eliminarian.

In [34]:
duplicate_sum = df_bosque.duplicated().sum()
if duplicate_sum > 0:
    print(f'Total de filas duplicadas: {duplicate_sum}')
else:
    print('No se encuentran filas duplicadas')

No se encuentran filas duplicadas


## Paso 1: Selección de variables basada en árbol de decisión

Para conseguir un dataset con una dimensión reducidad, aplica la técnica de Selección de variables basada en árbol de decisión mediante las importancias de cada variable (Decision Trees Importances):

Filtra el tablón para quedarnos solamente con las variables que aglutinan hasta el 95% de la información que se requiere para estimar la variable objetivo.
random_state=100

Primero hacemos una copia para poder guardar el dataset ya importado y no modificarlo.

In [35]:
df_bosque_f1 = df_bosque.copy()

A continuación, se deben de definir las variables objetivo y las predictoras. Tras ello mediante DecisionTreeRegresor para entrenar un modelo de decisión basado en los árboles de decisión. Al fijar como random_state 100 aunque el código se ejecute varias veces, se garantiza que el proceso de entrenamiento del modelo siempre se llevará a cabo de la misma manera.

In [36]:
target = 'Cover_Type'
features = [x for x in df_bosque_f1.columns if x!=target]

In [37]:
arbol = DecisionTreeRegressor(random_state=100)

El modelo se ajusta a los datos para poder predecir el valor de la variable objetivo a partir de las variables predictoras mediante .fit().

In [38]:
arbol.fit(X=df_bosque_f1[features], y=df_bosque_f1[target]) # Entrenamiento

DecisionTreeRegressor(random_state=100)

**VARIABLES IMPORTANTES**

In [39]:
importancias = arbol.feature_importances_ #Output: array con importancia de cada variable

Unicamente devuelve un array por tanto crearemos un Dataframe siendo los índices el nombre de las variables predictoras.

In [40]:
df_importancia = pd.DataFrame(arbol.feature_importances_, index=features, columns=["Importancia"])

df_importancia.sort_values(by=df_importancia.columns[0], ascending=False, inplace=True) #ORDENAR DATA FRAME: DECRECIENTE

df_importancia.head(10)

,Importancia
Elevation,0.345865
Horizontal_Distance_To_Fire_Points,0.173025
Horizontal_Distance_To_Roadways,0.170679
Horizontal_Distance_To_Hydrology,0.073564
Vertical_Distance_To_Hydrology,0.063559
Hillshade_3pm,0.041758
Hillshade_Noon,0.036356
Aspect,0.035264
Hillshade_9am,0.034812
Slope,0.025117


A este dataframe se le añade la columna con la importancia acumulada para de la siguiente manera coger los parámetros que contengan el 95% de la información.

In [41]:
df_importancia["imp_acum"] = df_importancia["Importancia"].cumsum()
df_importancia

,Importancia,imp_acum
Elevation,0.345865,0.345865
Horizontal_Distance_To_Fire_Points,0.173025,0.518890
Horizontal_Distance_To_Roadways,0.170679,0.689569
Horizontal_Distance_To_Hydrology,0.073564,0.763133
Vertical_Distance_To_Hydrology,0.063559,0.826692
Hillshade_3pm,0.041758,0.868450
Hillshade_Noon,0.036356,0.904806
Aspect,0.035264,0.940071
Hillshade_9am,0.034812,0.974883
Slope,0.025117,1.000000


In [42]:
df_importancia.loc[df_importancia['imp_acum']<=0.95] #95% IMPORTANCIA ACUMULADA

,Importancia,imp_acum
Elevation,0.345865,0.345865
Horizontal_Distance_To_Fire_Points,0.173025,0.518890
Horizontal_Distance_To_Roadways,0.170679,0.689569
Horizontal_Distance_To_Hydrology,0.073564,0.763133
Vertical_Distance_To_Hydrology,0.063559,0.826692
Hillshade_3pm,0.041758,0.868450
Hillshade_Noon,0.036356,0.904806
Aspect,0.035264,0.940071


Finalmente, las variables no importantes son las siguientes, que posteriormente serán eliminadas.

In [55]:
variables = df_importancia.loc[df_importancia['imp_acum']>0.95].index
variables = list(variables)
print('Variables no importantes: ',variables)
print('Total de variables tras la eliminación: ',len(features) - len(variables) + 1) # +1 por la variable objetivo

Variables no importantes:  ['Hillshade_9am', 'Slope', 'Wilderness_Area', 'Soil_Type']
Total de variables tras la eliminación:  9


Tras ello, obtenemos nuestro dataset reducido con las nuevas variables.

In [56]:
df_bosque_f2 = df_bosque_f1.drop(labels=variables, axis='columns')
df_bosque_f2.head()

,Elevation,Aspect,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2596,51,258,0,510,232,148,6279,5
1,2590,56,212,-6,390,235,151,6225,5
2,2804,139,268,65,3180,238,135,6121,2
3,2785,155,242,118,3090,238,122,6211,2
4,2595,45,153,-1,391,234,150,6172,5
